In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential 
from keras.layers import Dense, Embedding, LSTM, Dropout, Masking
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
train = pd.read_csv('data/train.csv')
train.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
0,2021-09-01 00:00:00+03:00,11.2,10.3,94.0,NaN,NaN,320.0,7.2,16.7,1012.6,2.0,0.09016,0.577
1,2021-09-01 01:00:00+03:00,10.7,9.6,93.0,NaN,NaN,320.0,7.2,13.0,1012.6,2.0,0.09251,0.594
2,2021-09-01 02:00:00+03:00,9.9,9.0,94.0,NaN,NaN,320.0,7.2,13.0,1012.2,2.0,0.08890,0.685
3,2021-09-01 03:00:00+03:00,10.0,8.4,90.0,NaN,NaN,330.0,7.2,13.0,1011.9,1.0,0.08735,1.016
4,2021-09-01 04:00:00+03:00,9.0,8.1,94.0,NaN,NaN,300.0,3.6,13.0,1011.4,2.0,0.08688,0.677


In [3]:
test = pd.read_csv('data/test.csv')
test.head()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price
0,2022-08-25 00:00:00+03:00,21.1,15.2,69.0,0.0,NaN,340.0,9.0,9.3,1022.0,2.0,0.25533
1,2022-08-25 01:00:00+03:00,20.1,15.1,73.0,0.0,NaN,30.0,6.0,14.8,1022.0,2.0,0.19492
2,2022-08-25 02:00:00+03:00,20.1,15.1,73.0,0.0,NaN,320.0,7.0,13.0,1022.0,2.0,0.18853
3,2022-08-25 03:00:00+03:00,18.7,17.0,90.0,0.0,NaN,0.0,4.0,11.1,1022.4,4.0,0.19947
4,2022-08-25 04:00:00+03:00,18.1,17.1,94.0,0.0,NaN,280.0,7.0,11.1,1022.0,3.0,0.21192


In [4]:
# Count missing values
train.isna().sum()

time              0
temp              0
dwpt              0
rhum              0
prcp           6433
snow           8473
wdir              0
wspd              0
wpgt              0
pres              0
coco            196
el_price          0
consumption       2
dtype: int64

In [5]:
def set_time_index(df):
  # [1444, 4972] - iloc, where time is changing
  df['time'] = pd.to_datetime(df['time'], utc=True).dt.tz_convert('Europe/Tallinn')
  df.set_index('time', drop=True, inplace=True)
  return df

In [6]:
train = set_time_index(train)
train.drop('snow', axis=1, inplace=True)
train.head()

,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,coco,el_price,consumption
time,,,,,,,,,,,
2021-09-01 00:00:00+03:00,11.2,10.3,94.0,NaN,320.0,7.2,16.7,1012.6,2.0,0.09016,0.577
2021-09-01 01:00:00+03:00,10.7,9.6,93.0,NaN,320.0,7.2,13.0,1012.6,2.0,0.09251,0.594
2021-09-01 02:00:00+03:00,9.9,9.0,94.0,NaN,320.0,7.2,13.0,1012.2,2.0,0.08890,0.685
2021-09-01 03:00:00+03:00,10.0,8.4,90.0,NaN,330.0,7.2,13.0,1011.9,1.0,0.08735,1.016
2021-09-01 04:00:00+03:00,9.0,8.1,94.0,NaN,300.0,3.6,13.0,1011.4,2.0,0.08688,0.677


In [7]:
test = set_time_index(test)
test.drop('snow', axis=1, inplace=True)
test.head()

,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,coco,el_price
time,,,,,,,,,,
2022-08-25 00:00:00+03:00,21.1,15.2,69.0,0.0,340.0,9.0,9.3,1022.0,2.0,0.25533
2022-08-25 01:00:00+03:00,20.1,15.1,73.0,0.0,30.0,6.0,14.8,1022.0,2.0,0.19492
2022-08-25 02:00:00+03:00,20.1,15.1,73.0,0.0,320.0,7.0,13.0,1022.0,2.0,0.18853
2022-08-25 03:00:00+03:00,18.7,17.0,90.0,0.0,0.0,4.0,11.1,1022.4,4.0,0.19947
2022-08-25 04:00:00+03:00,18.1,17.1,94.0,0.0,280.0,7.0,11.1,1022.0,3.0,0.21192


In [8]:
def get_corr_consumption(df):
  df_corr = df.corr(method="pearson")
  print(df_corr.shape)
  print("Correlation with consumption:")
  return pd.DataFrame(df_corr["consumption"].sort_values(ascending=False))

In [9]:
df_corr_train_consumption = get_corr_consumption(train)
df_corr_train_consumption

(11, 11)
Correlation with consumption:


,consumption
consumption,1.000000
coco,0.133539
rhum,0.094408
wpgt,0.072206
wspd,0.048840
wdir,-0.005009
prcp,-0.010668
pres,-0.068940
el_price,-0.124740
dwpt,-0.250484


In [10]:
def make_lag(df, column, n_lag=1):
  return df[column].shift(n_lag)

In [11]:
def train_val_split(df, vaL_days_size=7, train_days_size=None):
  vaL_days_size = vaL_days_size * 24
  if train_days_size != None:
    train_days_size = train_days_size * 24
    val_X = df.iloc[df.shape[0]-vaL_days_size:].drop(columns=['consumption'])
    val_y = df.iloc[df.shape[0]-vaL_days_size:]['consumption']
    train_X = df.iloc[df.shape[0]-(vaL_days_size + train_days_size):df.shape[0]-vaL_days_size].drop(columns=['consumption'])
    train_y = df.iloc[df.shape[0]-(vaL_days_size + train_days_size):df.shape[0]-vaL_days_size]['consumption']
    return train_X, train_y, val_X, val_y
  else:
    val_X = df.iloc[df.shape[0]-vaL_days_size:].drop(columns=['consumption'])
    val_y = df.iloc[df.shape[0]-vaL_days_size:]['consumption']
    train_X = df.iloc[:df.shape[0]-vaL_days_size].drop(columns=['consumption'])
    train_y = df.iloc[:df.shape[0]-vaL_days_size]['consumption']
    return train_X, train_y, val_X, val_y

In [12]:
train_X, train_y, val_X, val_y = train_val_split(train, 7, 28)

In [13]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler()

train_X_scaled = X_scaler.fit_transform(train_X)
train_X = pd.DataFrame(train_X_scaled, columns = train_X.columns, index=train_X.index)

val_X_scaled = X_scaler.transform(val_X)
val_X = pd.DataFrame(val_X_scaled, columns = val_X.columns, index=val_X.index)

In [14]:
y_scaler = MinMaxScaler()

train_y_scaled = y_scaler.fit_transform(train_y.values.reshape(-1, 1))
train_y = pd.Series(train_y_scaled.ravel(), name = train_y.name, index=train_y.index)

val_y_scaled = y_scaler.transform(val_y.values.reshape(-1, 1))
val_y = pd.Series(val_y_scaled.ravel(), name = val_y.name, index=val_y.index)

# inverse = scaler.inverse_transform(normalized)

In [15]:
for_lags = pd.concat([train_X, val_X]).join(train_y)

In [16]:
for_lags = for_lags.astype('float32')

In [17]:
columns_for_lags = ['consumption', 'temp', 'dwpt', 'el_price']
lags = [1, 2, 3, 12, 24, 48, 72]
for column_name in columns_for_lags:
  for lag in lags:
    for_lags[f'{column_name}_l{lag}'] = make_lag(for_lags, column_name, lag)

for_lags = for_lags.astype('float32')        
for_lags.drop(index=for_lags.index[:max(lags)], axis=1, inplace=True)

In [18]:
train_X, train_y, val_X, _ = train_val_split(for_lags, 7)

In [19]:
train_X = train_X.astype('float32')
train_y = train_X.astype('float32')
val_X = val_X.astype('float32').fillna(-1)

In [20]:
train_X = train_X.values.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.values.reshape((val_X.shape[0], 1, val_X.shape[1]))

In [21]:
verbose = 2
epochs = 100
batch_size = 72
n_timesteps, n_features, n_outputs = train_X.shape[1], train_X.shape[2], val_y.shape[0]

In [23]:
model = Sequential()
# model.add(Masking(mask_value=0., input_shape=(n_timesteps, n_features)))
model.add(LSTM(50, activation='relu', input_shape=(n_timesteps, n_features), return_sequences=True))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_outputs))
# model.compile(loss='mse',optimizer='adam', metrics=[tf.keras.metrics.MeanAbsoluteError()])
model.compile(loss='mse',optimizer='Adam')

# fit network
history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(val_X, val_y), verbose=verbose, shuffle=False)

# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

Epoch 1/100


ValueError: in user code:

    File "c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\Ivan\anaconda3\envs\ML\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_ob

In [ ]:
from sklearn.metrics import mean_absolute_error
val_pred = model.predict(val_X)

In [ ]:
val_pred[0]

In [ ]:
compare = pd.DataFrame([val_pred[0].ravel(), val_y.ravel()]).T.set_index(val_y.index)
compare.columns = ['pred', 'true']
compare.plot()
print(f'MAE = {mean_absolute_error(val_y, val_pred[0])}')